In [ ]:
import torch as tc
from typing import Union
import pathlib
import librosa


class Loader(ABC):
    @abstractmethod
    def load_case(self, input_path: Union[str, pathlib.Path], loading_params: dict) -> tc.Tensor:
        pass

    def get_loading_params(self) -> dict:
        return {}


class RawLoader(Loader):
    def __init__(self, sample_rate: int = 44100):
        self.sample_rate = sample_rate

    def load_case(self, input_path: Union[str, pathlib.Path], loading_params: dict) -> tc.Tensor:
        waveform, _ = librosa.load(input_path, sr=self.sample_rate, mono=False, offset=loading_params['offset'], duration=0.3)
        if waveform.shape[0] == 2:
            tensor = tc.from_numpy(waveform)
        elif waveform.shape[0] == 1:
            tensor = tc.from_numpy(waveform).repeat(2, 1)
        else:
            raise ValueError("Nieobsługiwana liczba kanałów w pliku dźwiękowym.")

        tensor = tensor.unsqueeze(0)

        return tensor


class SpectrogramLoader(Loader):
    def __init__(self, sample_rate: int = 44100, n_fft: int = 2048, hop_length: int = 512, stereo: bool = False):
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.stereo = stereo

    def load_case(self, input_path: Union[str, pathlib.Path], loading_params: dict) -> tc.Tensor:
        waveform, _ = librosa.load(input_path, sr=self.sample_rate, mono=not self.stereo, offset=loading_params['offset'], duration=0.3)

        spectrogram = tc.stft(tc.from_numpy(waveform), n_fft=self.n_fft, hop_length=self.hop_length, center=True)

        return tc.abs(spectrogram)

    def get_loading_params(self) -> dict:
        return {
            "sample_rate": self.sample_rate,
            "n_fft": self.n_fft,
            "hop_length": self.hop_length,
            "stereo": self.stereo
        }


class MelspectrogramLoader(Loader):
    def __init__(self, sample_rate: int = 44100, n_fft: int = 2048, hop_length: int = 512, n_mels: int = 128, stereo: bool = False):
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.stereo = stereo

    def load_case(self, input_path: Union[str, pathlib.Path], loading_params: dict) -> tc.Tensor:
        waveform, _ = librosa.load(input_path, sr=self.sample_rate, mono=not self.stereo, offset=loading_params['offset'], duration=0.3)

        mel_spec = librosa.feature.melspectrogram(waveform, sr=self.sample_rate, n_fft=self.n_fft, hop_length=self.hop_length, n_mels=self.n_mels)

        return tc.from_numpy(mel_spec)

    def get_loading_params(self) -> dict:
        return {
            "sample_rate": self.sample_rate,
            "n_fft": self.n_fft,
            "hop_length": self.hop_length,
            "n_mels": self.n_mels,
            "stereo": self.stereo
        }
